In [2]:
# %load_ext autoreload
# %autoreload 2
import time
import random
# import geopandas as gpd
import networkx as nx
#import matplotlib.pyplot as plt
import numpy as np
import math
import pandas as pd
# from shapely.geometry import Point
from matplotlib import animation
from JSAnimation import IPython_display
# %matplotlib inline
import copy
import matplotlib.pylab as plt
from itertools import islice
import csv
import simpy 




## Generate Graph

In [3]:



import time
start_time = time.time()


def calculate_distance(Node1, Node2):
    # calculate distance between two points
    distance = float(((Node2[0]-Node1[0])**2+(Node2[1]-Node1[1])**2)**0.5)
    return float(distance)

G=nx.read_shp('graphs/Amsterdam-Canals/final_network_v4.shp', simplify=True)
#G = G.to_undirected()
#G = max(nx.connected_component_subgraphs(G), key=len)

H_G = nx.Graph()

lat_lon_to_index = {}
edge_id_counter = 0

for i, node in enumerate(G.nodes()): #adds nodes with label 1 till n nodes to the network. 
    H_G.add_node(i, pos= node)
    lat_lon_to_index[node] = i

    
for edge in G.edges(data=True): # adds edges with attributes to the graph from shapefile according to the order in which the edges are ordered in the shapefile
    H_G.add_edge(lat_lon_to_index[edge[0]], lat_lon_to_index[edge[1]], visited=0, edge_id = edge_id_counter, turnaround =edge[2]['turnaround'], width= edge[2]['width'], rcv_visted = 0, cv_visted = 0, dis= calculate_distance(nx.get_node_attributes(H_G, 'pos')[lat_lon_to_index[edge[1]]], nx.get_node_attributes(H_G, 'pos')[lat_lon_to_index[edge[0]]]) )
    edge_id_counter +=1
H_G.add_edge(30,26, width = 50, rcv_visted = 0, edge_id = 444, cv_visted = 0, turnaround= 1,  visited = 0,  dis= calculate_distance(nx.get_node_attributes(H_G, 'pos')[30], nx.get_node_attributes(H_G, 'pos')[26] ))


w_G = H_G.to_directed()

w_G.edges[236,258]['turnaround'] =1
w_G.edges[258,236]['turnaround'] =1
w_G.edges[258,259]['turnaround'] =1
w_G.edges[259,258]['turnaround'] =1
w_G.edges[303,285]['turnaround'] =1
w_G.edges[285,303]['turnaround'] =1
w_G.edges[285,236]['turnaround'] = 1
w_G.edges[236,285]['turnaround'] = 1

#make one-way traffic on certain edges:
#prinsengracht
w_G.remove_edge(352,351)
w_G.remove_edge(353,352)
w_G.remove_edge(296,353)
w_G.remove_edge(297,296)
w_G.remove_edge(280,297)
w_G.remove_edge(249,280)
w_G.remove_edge(250,249)
w_G.remove_edge(251,250)
w_G.remove_edge(201,251)
w_G.remove_edge(202,201)
w_G.remove_edge(203,202)
w_G.remove_edge(142,203)
w_G.remove_edge(121,142)
w_G.remove_edge(122,121)
w_G.remove_edge(85,122)
w_G.remove_edge(88,85)
w_G.remove_edge(31,88)
w_G.remove_edge(34,31)
w_G.remove_edge(35,34)
w_G.remove_edge(20,35)
w_G.remove_edge(36,20)
w_G.remove_edge(37,36)
w_G.remove_edge(38,37)
w_G.remove_edge(39,38)

#eenhoornssluis
w_G.remove_edge(351,367)
w_G.remove_edge(367,368)
w_G.remove_edge(368,369)
w_G.remove_edge(369,366)
w_G.remove_edge(366,365)


#grimburgerwal
w_G.remove_edge(150,149)

#singelgracht
w_G.remove_edge(5,4)
w_G.remove_edge(4,6)
w_G.remove_edge(6,3)
w_G.remove_edge(3,2)

#eenhoornsluis & haarlemmersluis
w_G.remove_edge(351,354)
w_G.remove_edge(354,338)







#nodelist omzetten naar pandas dataframe omdat zodat hij veel makkelijker en sneller de coordinaten van de nodes kan opzoeken
#pandas >>> nx.get_node_attributes kwa snelheid



graph_df = pd.DataFrame.from_dict(nx.get_node_attributes(w_G, 'pos'), orient='index', columns=["lat", "long"]) 






print("Simulation : {} s".format(time.time() - start_time))

Simulation : 0.2712380886077881 s


In [4]:
w_G.edges[105,106]['dis']

122.220319461464

### Gracht-werkzaamheden scenario's

In [5]:
scenario_G= w_G.copy()
#CHECK ALWAYS THE ROUTES; 


# #tests

# scenario_G.remove_edge(87,86)
# scenario_G.remove_edge(86,87)

# scenario_G.remove_edge(228,229)
# scenario_G.remove_edge(229,228)

# scenario_G.remove_edge(121,122)
# scenario_G.remove_edge(122,121)

# w_G.remove_edge(352,353)
# w_G.remove_edge(231,230)
# w_G.remove_edge(230,231)
# w_G.remove_edge(123,124)
# w_G.remove_edge(124,123)

# w_G.remove_edge(125,124)
# w_G.remove_edge(124,125)

# w_G.remove_edge(336,337)
# w_G.remove_edge(337,338)

scenario_G_draw = scenario_G.to_undirected()


In [6]:
# plt.figure(figsize=(18,18))
# nx.draw(w_G, nx.get_node_attributes(w_G, 'pos'), with_labels=True, node_size=0.5, font_size = 2, width = 0.2, arrowsize = 3)
# plt.savefig('/Users/JeroenvanderDoes/Downloads/Amsterdam_Canals_lates_HAHA.pdf', format='pdf', rasterized=False)
# plt.show()

## Path generator; Random Tour Vessels

In [7]:
#meer startpunten
dict_start_points={
    #stromma & andere hotspots voor chartervaart
    'prinsengracht' : 121,
    'rijksmuseum':63,
    'annefrank': 280,
    'damrak': 316,
    'stationsplein': 319,
    #salon schepen
    'singelgracht 1': 290,
    'singelgracht 2':291,
    'oude turfmarkt':119,
    'oude schans1 ': 204,
    'waalseilandsgrahct': 286,
    'prinsengracht onder': 35

}

#meer wayoints
dict_waypoints_pleasurecraft ={
    'annefrank' : [296],
    'magerebrug': [56],
    'redlightdistrict' : [268],

    'prinsengracht 1' : [250],
    'prinsengracht 2' : [202],
    'prinsengracht 5': [121],
    'prinsengracht 3' : [122], #misschien veranderen met startpunt
    'prinsengracht 4' : [35],
    'prinsengracht 6' : [353],

    'artis 1' : [114],
    'artis 2' : [113],

    'rijksmuseum' : [10],
    'stnicolaasbassilica' : [302],
    'scheepsvaartmuseum' : [192], #uitzetten voor pleasurecraft
    'rembrandhuisenwaterlooplein' : [147],
    'st antonie sluis': [193],
    'st antonie sluis2': [194],

    'keizersgracht boven 1':[337],
    'keizersgracht boven 2':[336],

    'keizersgracht midden & negenstraatjes 1':[238],
    'keizersgracht midden & negenstraatjes 2':[230],
    # 'keizersgracht midden & negenstraatjes 3':[335],

    'keizersgracht beneden 1':[109],
    'keizersgracht beneden 2':[107],

    'heerengracht boven': [311],
    'heerengracht midden 1':[248],
    'heerengracht midden 2': [246],
    'heerengracht beneden (goudenbocht) 1' : [126],
    'heerengracht beneden (goudenbocht) 2' : [127],

    'singel boven': [329],
    'singel midden 1' :[293],
    'singel midden 2': [290],


    'singelgracht boven 1':[360],
    'singelgracht boven 2':[358],
    'singelgracht midden 1':[224],
    'singelgracht midden 2':[101],
    'singelgracht onder 1' :[6],
    'singelgracht onder 2':[4],
    #test
    'singelgracht onder 3': [111],
    # 'singelgracht onder 4': [90],
    'oude centrum 1': [168],
    'oude centrum 2': [167],
    'oude centrum 3': [173],
    # 'oude centrum 4': [174],
    'oude centrum 5': [207],
    'oude schans': [204],
    "prinshendrikkade ":[288],
    'jordaan 1':[130],
    'jordaan 3':[143],
    'jordaan 4':[144],
    'jordaan 5': [199],
    'jordaan 6': [255],
    'jordaan 7': [305],
    'amstelsluizen': [28],
    'singelgracht':[6],
    'noordelijk jordaan':[377]
}

#kleinere zones
dict_zones = {
#must
    'annefrank' : [296],
    'magerebrug': [56],
    'rijksmuseum' : [10],
#     'redlightdistrict' : [268],
    'amstelsluizen': [28],

    #zone 1; links boven
    'zone 1; links boven':[337, 336 , 353, 305, 311, 329],


    #zone 2; links midden
    'zone 2; links midden' : [250, 202, 122, 238, 230, 335, 291, 130, 143, 144, 199, 255, 248, 292 ],


    #zone 3 beneden
    'zone 3; beneden':[109, 107, 126, 127],

    #zone 4 centrum
    'zone ;4 centrum': [168,167, 173, 174, 207, 204, 268, 236],

    #zone 5; rechts boven
    'zone 5; rechts boven' : [302, 192, 288],


    #zone 6: overig
    'singelgracht':[6, 114, 113]

}


## Vessel Base Class

In [8]:
class Vessel(object):
    def __init__(self, env, step_size, step_iter, id_vessel):
        """ Base class vessel, can sail on its own"""
        self.env = env
        self.id_vessel = id_vessel
        self.action = env.process(self.run())
        self.wait_time_list =[]
        self.total_wait_time = 0
        
        #old code
        self.draw = random.sample(w_G.nodes, 2)
        self.path = self.route(w_G,self.draw[0], self.draw[1])
        self.start = self.path[0]
        self.end = self.path[len(self.path)-1]
        self.coor_log = []
        self.vessel_pos = (graph_df['lat'].loc[self.path[0]], graph_df['long'].loc[self.path[0]]) # nx.get_node_attributes(w_G, 'pos')[self.path[0]] 
        self.w_G = w_G
        self.to_node_id = 1
        self.move = 3 #m per stap
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]
        #self.start_time = simulation.step_count
        
        #moet uit sim komen
        self.step_size = step_size
        self.step_iter = step_iter
        self.wait_node_list =[]
        
        self.velocity = 5 # m/s
        
        # deze meuk is om de snelheid op elke link te berekenen na de run
        self.from_list = []
        self.to_list = []
        self.dist_list = []
        self.time_list = []
        self.step_list = []
        self.function_log_list = []
        self.queue_list = []
        self.type = "baseclass"
        
    def run(self):
        while True:
            if self.active == True:
                to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                distance_to_next_node = self.calculate_distance(self.vessel_pos, to_node_pos)
                if self.path[self.to_node_id] == self.path[ len(self.path)-1] and distance_to_next_node < self.move:
                        self.vessel_pos = self.path[len(self.path)-1]
                        self.active = False
#                         self.log_step(distance_to_next_node, 2)
                else:
                    to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                    distance_to_next_node = self.calculate_distance(self.vessel_pos, to_node_pos)
                    if self.path[self.to_node_id] in dict1.keys() and distance_to_next_node < self.move:
                        t_arrival = env.now
                        with dict1[self.path[self.to_node_id]].request() as request:
                            yield request
                            wait_total = int(env.now - t_arrival)
                            if wait_total > 0:
                                self.wait_time_list.append(wait_total)
                                self.total_wait_time += wait_total
                                for i in range(wait_total):
                                    # voor visualisatie mbt animeren moeten deze stappen gelogd worden
                                    self.coor_log_x.append(self.vessel_pos[0]) 
                                    self.coor_log_y.append(self.vessel_pos[1])
                                    # 1 staat voor wachten
                                    self.log_waiting_step(10, 1)
                            for i in range(6): #gemiddeld 1 minuut om over te steken
                                    self.coor_log_x.append(self.vessel_pos[0]) 
                                    self.coor_log_y.append(self.vessel_pos[1])
                                    # 2 staat voor gebruiken van kruispunt
                                    self.log_waiting_step(10, 2)
                                    yield env.timeout(1)
                    self.move_on_path() # moet deze niet onderaan?
            sail_step_duration = 1
            yield self.env.timeout(sail_step_duration)
        
        
        
    def route(self, w_G, start, end):
        return nx.dijkstra_path(w_G, start, end, weight = 'dis')
    
    
    def calculate_distance(self, Node1, Node2):
        # calculate distance between two points
        distance = float(((Node2[0]-Node1[0])**2+(Node2[1]-Node1[1])**2)**0.5)
        return float(distance)
    

    def calculate_angle(self, Node1, Node2):
        # calculate angle going from point 1 to point 2 (NB: atan2(y,x))
        angle =  (float(math.atan2(Node2[1]-Node1[1],Node2[0]-Node1[0]))) #*float((180.0 / math.pi)) for degrees
        return float(angle)
        

    def pos_next_step(self, Node1, Node2, move):#move cannot be replaced by self.move because move may change by 'remaining_move'
        # move towards Node2
        distance = self.calculate_distance(Node1, Node2) #wordt dit wel hier gebruikt?
        angle = self.calculate_angle(Node1, Node2)
        new_node = (float(Node1[0]+math.cos(angle)*move), float(Node1[1]+math.sin(angle)*move))
        return new_node
    
    def log_step(self, distance_to_next_node, log_nr, laststep=True):
        if self.path[self.to_node_id - 1] != self.path[self.to_node_id]:
            if laststep == True:
                writer.writerow([self.id_vessel, \
                                 self.path[self.to_node_id - 1],\
                                 self.path[self.to_node_id],\
                                 distance_to_next_node,\
                                 self.step_size * (distance_to_next_node / self.move),\
                                 self.step_iter,\
                                 self.type,\
                                 log_nr,\
                                 0,\
                                 w_G.edges[self.path[self.to_node_id - 1], self.path[self.to_node_id]]["edge_id"],\
                                 self.step_iter/(3600/self.step_size),\
                                 self.to_node_id,\
                                 self.vessel_pos[0],\
                                 self.vessel_pos[1],
                                 w_G.edges[self.path[self.to_node_id - 1] , self.path[self.to_node_id]]['dis']])
                
            elif laststep == False:
                writer.writerow([self.id_vessel, \
                                 self.path[self.to_node_id - 1],\
                                 self.path[self.to_node_id],\
                                
                                 distance_to_next_node,\
                                 self.step_size * (distance_to_next_node / self.move),\
                                 self.step_iter,\
                                 self.type,\
                                 log_nr,\
                                 0,\
                                 w_G.edges[self.path[self.to_node_id - 1], self.path[self.to_node_id]]["edge_id"],\
                                 self.step_iter/(3600/self.step_size),\
                                 self.to_node_id,\
                                 np.nan,\
                                 np.nan,\
                                 w_G.edges[self.path[self.to_node_id - 1] , self.path[self.to_node_id]]['dis']])
        
    def log_waiting_step(self, log_nr, wait_or_use):
        if self.path[self.to_node_id - 1] != self.path[self.to_node_id]:
            writer.writerow([self.id_vessel,\
                             self.path[self.to_node_id - 1],\
                             self.path[self.to_node_id],\
                             0,\
                             self.step_size,\
                             self.step_iter,\
                             self.type,\
                             log_nr,\
                             wait_or_use,\
                             w_G.edges[self.path[self.to_node_id - 1], self.path[self.to_node_id]]["edge_id"],\
                             env.now/(3600/self.step_size),\
                             self.to_node_id,\
                             self.vessel_pos[0],\
                             self.vessel_pos[1],
                             w_G.edges[self.path[self.to_node_id - 1] , self.path[self.to_node_id]]['dis']])
    
    def move_on_path(self):
        if self.type == "cruise":
            self.velocity_list.append(self.move)
        # 1. vessel_pos is your boat
        # 2. you need to know the next node in the path
        # 3. calculate the distance between Node1 and the next node
        
        if self.active == True: #start logging  each step of vessel;
            
            to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
            
            distance_to_next_node = self.calculate_distance(self.vessel_pos, to_node_pos)

            # If move <= distance to next node, make move
            if self.move < distance_to_next_node:
                    vessel_pos_new = self.pos_next_step(self.vessel_pos, to_node_pos, self.move)
        
                    self.log_step(self.move, 1, laststep = True) 
                    
            else:
                if self.to_node_id == len(self.path)-1:
                    self.active = False #stop logging if vessel arrives at destination
                    vessel_pos_new = to_node_pos  
                
                    self.log_step(distance_to_next_node, 2, laststep = True) 
                    
                else:
                    self.log_step(distance_to_next_node, 3, laststep = False) 
                    
                    # reposition vessel at next node and calculate remaining move distance
                    self.vessel_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])

                    # up the to_node_id with 1 and find new to_node_pos
                    self.to_node_id+=1
                    
                    # calculate distance to next node
                    remaining_move = float(self.move-distance_to_next_node)
                    
                    #updates new distance to next node
                    to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                    distance_to_next_node = self.calculate_distance(self.vessel_pos, to_node_pos)
                    
                    #checks for 'overshoot', if remaining distance > next distance, place on next node and check again till condition is no longer valid
                    while remaining_move > distance_to_next_node:
                        
                        # if statement to check if it is on end of path (last point of path could be couple of nodes which lengt < move)
                        #otherwise, node_id > len of path and can;t compute
                        if self.to_node_id == len(self.path)-1:
                            self.active = False #stop logging if vessel arrives at destination
                            break
                        
                        self.log_step(distance_to_next_node, 4, laststep = False)
                        self.vessel_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                        remaining_move = float(remaining_move-distance_to_next_node)
                
                        self.to_node_id+=1
                        
                        #vessel is placed on new node, now update distance to next node 
                        to_node_pos = (graph_df['lat'].loc[self.path[self.to_node_id]], graph_df['long'].loc[self.path[self.to_node_id]])
                        distance_to_next_node = self.calculate_distance(self.vessel_pos, to_node_pos)
                        #end of loop

                    if self.active == True:
                        self.log_step(remaining_move, 6, laststep = True)
                    elif self.active == False:
                        self.log_step(distance_to_next_node, 7, laststep = True)
                        
                    vessel_pos_new = self.pos_next_step(self.vessel_pos, to_node_pos, remaining_move)
                    
            #updates position of vessel
            self.vessel_pos = vessel_pos_new
            
            #adds new positions to log
#             self.coor_log.append(self.vessel_pos) #library for tuple (x,y) coordinate of the vessel
            self.coor_log_x.append(self.vessel_pos[0]) # library for only x coordinates
            self.coor_log_y.append(self.vessel_pos[1])# library for only y coordinates
            
            self.step_iter += 1
            


## Vessel sub-categories

In [9]:
class Cruise_Vessel(Vessel):
    
    def __init__(self, env,  step_size, step_iter, id_vessel):
        """Overwrites __init__ from baseclass to specify specific vessel"""
        #simpy code
        self.env = env
        self.action = env.process(self.run())
        self.wait_time_list =[]
        self.total_wait_time = 0
        
        #moet uit sim komen
        self.step_size = step_size
        self.step_iter = step_iter
        self.id_vessel = id_vessel
        
        #calculates stepsize of vessel
        self.velocity = np.random.uniform(5.5,7,1)[0] # velocity differs from 5.5 till 7 km/h
        self.velocity_list = []
        self.move = (self.velocity/3.6)*self.step_size
        
        #old code
        self.dict_routes = self.routes_maker()
        self.key = random.sample(list(self.dict_routes), 1) #can be input from simulator.
        self.path = self.route(self.key[0]) 
        self.coor_log = []
        self.vessel_pos = (graph_df['lat'].loc[self.path[0]], graph_df['long'].loc[self.path[0]]) #start position
        self.to_node_id = 1
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]
        self.wait_node_list =[]
        
        
        # deze meuk is om de snelheid op elke link te berekenen na de run
        self.from_list = []
        self.to_list = []
        self.dist_list = []
        self.time_list = []
        self.step_list = []
        self.function_log_list = []
        self.queue_list = []
        self.type = "cruise"
        
        
    def routes_maker(self):
        cruise_vessel_routes_pd = pd.read_excel('/Users/JeroenvanderDoes/Downloads/schema_2.xlsx', index_col = 'NAAM')
        cruise_vessel_routes_pd = cruise_vessel_routes_pd.fillna("empty")

        dict_waypoints={}
        for i in cruise_vessel_routes_pd.index.tolist():
            dict_waypoints[i] = [int(x) for x in cruise_vessel_routes_pd.loc[:, 'NODES':].loc[i].tolist() if x != 'empty']

        dict_routes = {}
        for i in dict_waypoints:
            route = []
            total_route = []
            for x, j in enumerate(dict_waypoints[i][:len(dict_waypoints[i]) - 1]):
                l = nx.dijkstra_path(w_G,dict_waypoints[i][x] , dict_waypoints[i][x+1], weight = 'dis')
                route.append(l[:-1])
            route.append([route[0][0]])
            for r in route:
                total_route += r
            dict_routes[i] = total_route

        return dict_routes

        
    def route(self, key):
        
        list1 = []
        scenario_network = scenario_G.copy()
        for i in scenario_network.edges(data=True):
            if i[2]['width'] < 13.0:
                list1.append((i[0], i[1]))
       
        scenario_network.remove_edges_from(list1)
        
         #low hanging bridge on heerengracht 
        scenario_network.remove_edge(54,55)
        scenario_network.remove_edge(55,54)

        
        route_test = self.dict_routes[key]
        route_check =[]
        route_check+= nx.dijkstra_path(scenario_network, route_test[-2],route_test[-1])[1:]
        for i in range(len(route_test) -1 ):
            if scenario_network.has_edge(route_test[i], route_test[i+1] )==True:
                route_check.append(route_test[i+1])

        alternative_path = route_check[:]



        #check if nodes can be reached, else it deletes nodes from path

        def route_checker(route_check):
            i=0
            while i < len(route_check)-1:

                if nx.has_path(scenario_network, route_check[i], route_check[i+1]) == False:
        #             print("node to be deleted", route_check[i+1] )
                    del route_check[i+1]

                # hij kan ook vast komen te zitten 
                else:
                    i+=1
            return route_check

        route_checker(route_check)       

        #check of laatste node kan worden bereikt, zo neit, dan zit ie vast en moet hij terug op het pad waarvan dat wel kan
        #scenario gebeurt alleen maar als je eenrichting hebt, dan kan ie iet omdraaien
        if route_check[0] != route_check[-1]:
            new_path = alternative_path
            j = len(route_check)
            p = scenario_network.has_edge(new_path[j-1], new_path[j-2])
            list_of_edges =[]
            list_of_edges.append(new_path[j-1])
            #checks untill where one way path is
            while p == False:
                j = j-1
                p = scenario_network.has_edge(new_path[j-1], new_path[j-2])
                list_of_edges.append(new_path[j-1])

            for i in list_of_edges[:-1]:
                new_path.remove(i)
            route_check = new_path
            #calculate new route from new checkpoint
            route_check = route_checker(route_check)    


        final_path =[]

        for i in range(len(route_check)-1):
            path = nx.dijkstra_path(scenario_network, route_check[i],route_check[i+1], weight = 'dis')
            final_path += path[:-1]

        for i in range(len(final_path[:-4])-1):
            scenario_network.edges[final_path[i], final_path[i+1]]['cv_visted'] += 1
            try:
                scenario_network.edges[final_path[i+1], final_path[i]]['cv_visted'] += 1
            except Exception:
                pass

        j = final_path[0]
        if final_path[0] != final_path[-1]:
            final_path.append(j)

        i = 0
        while i < len(final_path)-1:
            if scenario_network.edges[final_path[i], final_path[i+1]]['cv_visted'] >1 and\
            scenario_network.edges[final_path[i], final_path[i+1]]['turnaround'] ==0:
                print(final_path[i])
                del final_path[i]
            else:
                i += 1
                
        return final_path



In [10]:
class Pleasure_Craft(Vessel):
    def __init__(self, env, step_size, step_iter, id_vessel):
        """Overwrites __init__ from baseclass to specify specific vessel"""
    
        #simpy code
        self.env = env
        self.id_vessel = id_vessel
        self.action = env.process(self.run())
        self.wait_time_list =[]
        self.total_wait_time = 0
        
        #moet uit sim komen
        self.id_vessel = id_vessel
        self.step_size = step_size
        self.step_iter = step_iter
        self.type = "pleasure_craft"
        
        #total_round calculation
        self.round_duration = 1.25 #hr
        self.velocity = 6 # km/h
        self.move = (self.velocity/3.6)*self.step_size
        self.total_round = ((self.round_duration*60*60)/self.step_size)*self.move
        self.type = "random"
        
        #old code
        self.path = self.route()
        self.vessel_pos = (graph_df['lat'].loc[self.path[0]], graph_df['long'].loc[self.path[0]]) #start position
        self.w_G = w_G
        self.to_node_id = 1
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]
        self.coor_log = []
    
        self.wait_node_list =[]
    
        # deze meuk is om de snelheid op elke link te berekenen na de run
        self.from_list = []
        self.to_list = []
        self.dist_list = []
        self.time_list = []
        self.step_list = []
        self.function_log_list = []
        self.queue_list = []
        self.startpoint_reference = 0
     
    def route(self):
        
        start_time = time.time()
        total_round = self.total_round
        dict_attractions = dict_waypoints_pleasurecraft
        copy_w_G_totaal= scenario_G.copy() # keeps track on path actually visited as input for next iteration

        #output
        total_path =[]
        global startpoint
        startpoint = 0 

        def route_planner_captain(total_path,  total_round, dict_attractions, copy_w_G_totaal):
            
            c=list(w_G.nodes)
            startpoint_node = np.random.choice(c, 1)[0]
            
            node_from_here = startpoint_node
            startpoint = startpoint_node
            self.startpoint_reference = startpoint_node
            
            penalty = 1
            reference_total_round = total_round
            treshold = 100

            copy_sightseeing_dict = copy.deepcopy(dict_attractions)

            minimal_distance_next_attraction = 100000000000000
            final_distance_back = 0
            final_path_back =[]
            
            path = []
            

            dict_zones_copy = copy.deepcopy(dict_waypoints_pleasurecraft)
            
            
            list_names=[]
            list_nodes=[]
            self.next_point_zone =[]
            for i,v in zip(dict_zones_copy.values(), dict_zones_copy.keys()):
                list_nodes.append(i)
                list_names.append(v)
             
            j=0
            while total_round - final_distance_back > treshold:
                
                #line of code makes sure no neigboring nodes are drawn in order to prevent turn around.
                #makes copy of dictionary 
                copy_sightseeing_dict_for_drawing = copy.deepcopy(copy_sightseeing_dict)

                #exclusions

#                 if list_names[j] == 'redlightdistrict' or list_names[j] == 'zone ;4 centrum':
#                     list_checker = [x for x in list_nodes if x != [] and x != list_nodes[5]]
#                     print("list checker if", list_names[j], list_names[3], list_names[5])

#                 elif list_names[j] == 'zone 1; links boven':
#                     list_checker = [x for x in list_nodes if x != [] and x != list_nodes[3] and  x != list_nodes[4]]
#                     print("list checker elif", list_checker)

                list_checker = [x for x in list_nodes if x != []]
    
    
                if total_round == reference_total_round:
              
                    lis_nodes_not=[]
                    for i in c:
                        p = list(w_G.successors(i))
                        x = list(w_G.predecessors(i))
                        if len(p) == 1 and len(x) == 1 and p[0] == x[0]:
                            lis_nodes_not.append(i)
                        list_nodes =[i for i in list_checker if i not in lis_nodes_not]
                        next_point_zone = random.sample(list_nodes, 1)[0]
                        next_point_node= random.sample(next_point_zone, 1)
                else:

                    list_nodes = list_checker
                    next_point_zone = random.sample(list_nodes, 1)[0]
                    next_point_node= random.sample(next_point_zone, 1)
                                
                
                neighbors = [n for n in copy_w_G_totaal[next_point_node[0]]]
                
                
                def possibillity_checker(next_point_node, node_from_here, startpoint, copy_w_G_totaal, path):
                    
                    w_G_path =scenario_G.copy()
                    
                    if total_round < reference_total_round and w_G_path.has_edge(path[len(path)-1], path[len(path)-2]) == True:
                        w_G_path.remove_edge(path[len(path)-2], path[len(path) -1])
                        try:
                            w_G_path.remove_edge(path[len(path)-1], path[len(path) -2])
                        except Exception:
                            pass

                    if nx.has_path(w_G_path, node_from_here, next_point_node[0] )== True:
                        
                        path_1 = nx.dijkstra_path(w_G_path, node_from_here, next_point_node[0], weight = 'dis')
                        
                        if total_round < reference_total_round and w_G_path.has_edge(path_1[len(path_1)-1], path_1[len(path_1)-2]) == True:
                            w_G_path.remove_edge(path_1[len(path_1)-2], path_1[len(path_1) -1])
                            try:
                                w_G_path.remove_edge(path_1[len(path_1)-1], path_1[len(path_1) -2])
                            except Exception:
                                pass
                            
                        path_check = nx.has_path(w_G_path, next_point_node[0], startpoint)
                        
                    else:
                        path_check=False
                    return path_check
                
                
                possible_path = possibillity_checker(next_point_node, node_from_here, startpoint, copy_w_G_totaal, path)
                            
                is_path = nx.has_path(copy_w_G_totaal, node_from_here, next_point_node[0])
                
                while is_path == False or copy_w_G_totaal.edges[next_point_node[0], neighbors[0]]['visited'] > 0\
                or possible_path == False:
                    list_checker = [x for x in list_nodes if x != []]
                    list_nodes = list_checker
                    if len(list_nodes)==0:
                        break
                    next_point_zone = random.sample(list_nodes, 1)[0]
                    next_point_node= random.sample(next_point_zone, 1)
                    is_path = nx.has_path(copy_w_G_totaal, node_from_here, next_point_node[0])
                    possible_path = possibillity_checker(next_point_node, node_from_here, startpoint, copy_w_G_totaal, path)
                    neighbors = [n for n in copy_w_G_totaal[next_point_node[0]]]
    
                    j = list_nodes.index(next_point_zone)
                    list_nodes[j].remove(next_point_node[0])
        
            
                if len(list_nodes)==0:
                    total_round -= final_distance_back
                    for i in final_path_back:
                        total_path.append(i)
                    break
                
#                 j = list_nodes.index(next_point_zone)
#                 print("list_nodes", list_nodes, "index", j, "next_point_node", next_point_node )
#                 list_nodes[j].remove(next_point_node[0])

                
                j = list_nodes.index(next_point_zone)
                list_nodes.remove(next_point_zone)
                

                for attraction in next_point_node:
                    
                    copy_w_G_1 = copy_w_G_totaal.copy()
                    
                    #two edges
                    if total_round < reference_total_round and copy_w_G_1.has_edge(path[len(path)-2], path[len(path)-1]) == True\
                    and copy_w_G_1.has_edge(path[len(path)-1],path[len(path)-2]) == True:
                        copy_w_G_1.remove_edge(path[len(path)-2], path[len(path) -1])
                        try:
                            copy_w_G_1.remove_edge(path[len(path)-1], path[len(path) -2])
                        except Exception:
                            pass
                                  
                    distance_attraction = nx.dijkstra_path_length(copy_w_G_1, node_from_here, attraction , weight = 'dis')
                    path_next_attraction = nx.dijkstra_path(copy_w_G_1, node_from_here, attraction, weight = 'dis') 
                    
                    distance_attaction= 0
                    for i in range(len(path_next_attraction)-1):
                        copy_w_G_1.edges[path_next_attraction[i], path_next_attraction[i+1] ] ['dis'] *= penalty
                        try:
                            copy_w_G_1.edges[path_next_attraction[i+1], path_next_attraction[i] ] ['dis'] *= penalty
                        except Exception:
                            pass

                    ### om te voorkomen dat hij op het einde niet terug keert, werkt nog niet helemaal; paar uitzonderingen vandaar try except
                     #print("path_next_attraction", path_next_attraction)
            
                    if total_round < reference_total_round and copy_w_G_1.has_edge(path_next_attraction[len(path_next_attraction)-1],path_next_attraction[len(path_next_attraction)-2]) == True:
                        copy_w_G_1.remove_edge(path_next_attraction[len(path_next_attraction)-2], path_next_attraction[len(path_next_attraction) -1])
                        try:
                            copy_w_G_1.remove_edge(path_next_attraction[len(path_next_attraction)-1], path_next_attraction[len(path_next_attraction) -2])
                        except Exception:
                            pass
                    
                    
                    distance_attraction_back_home = nx.shortest_path_length(copy_w_G_1, attraction, startpoint, weight = 'dis')
                    path_back_home = nx.shortest_path(copy_w_G_1, attraction, startpoint, weight = 'dis')

                    if distance_attraction + distance_attraction_back_home < total_round:
                       
                        total_round -= distance_attraction
                        path = path_next_attraction
                        node_from_here  = attraction
                        final_path_back  = path_back_home
                        final_distance_back = distance_attraction_back_home
                    
                        for i in range(len(path)-1):
                            copy_w_G_totaal.edges[ path[i], path[i+1] ] ['dis'] *= penalty
                            copy_w_G_totaal.edges[ path[i], path[i+1] ] ['visited'] +=1
                            try:
                                copy_w_G_1.edges[path[i+1], path[i] ] ['dis'] *= penalty
                                copy_w_G_totaal.edges[ path[i+1], path[i] ] ['visited'] +=1
                            except Exception:
                                pass
              

                        #add path to total path
                        for i in path:
                            total_path.append(i)
                            #staan nog dubbele tussen
             
                if total_round - final_distance_back < treshold:

                    #hier heeft ie de laatste attractie nog niet weggehaald, wat wel gebeurd bij de vorige if
                    total_round -= final_distance_back
                    for i in final_path_back:
                        total_path.append(i)
              
                    break  

                elif len(list_nodes)==0:
                    total_round -= final_distance_back
                    for i in final_path_back:
                        total_path.append(i)
                    #print("list is done in while loop")
                    break 
                    
          

            return total_path#, total_percentage


        results= route_planner_captain(total_path, total_round, dict_attractions, copy_w_G_totaal)            
        route = results
        
        while len(route)==0:
            
            results= route_planner_captain(total_path, total_round, dict_attractions, copy_w_G_totaal)
            
        
        return route
     
    
    

In [11]:
class Random_Cruise_Vessel(Vessel):
    def __init__(self, env, step_size, step_iter, id_vessel):
        """Overwrites __init__ from baseclass to specify specific vessel"""
        #code from simpy
        self.env = env
        self.id_vessel = id_vessel
        self.action = env.process(self.run())
        self.wait_time_list =[]
        self.total_wait_time = 0
        
         #moet uit sim komen
        self.step_size = step_size
        self.step_iter = step_iter
        self.id_vessel = id_vessel
        
        #total_round calculation
        self.round_duration = 1.25 #hr
        self.velocity = 6 # km/h
        self.move = (self.velocity/3.6)*self.step_size
        self.total_round = ((self.round_duration*60*60)/self.step_size)*self.move
        self.type = "random"
        
        #route calculation
        self.path = self.route()
        self.vessel_pos = (graph_df['lat'].loc[self.path[0]], graph_df['long'].loc[self.path[0]]) #start position
        self.w_G = w_G
        self.to_node_id = 1
        self.travel_time = 0
        self.active = True
        self.coor_log_x =[]
        self.coor_log_y=[]
        self.coor_log = []
        
        #log functions; i think old because everything will be directly written to csv
        self.from_list = []
        self.to_list = []
        self.dist_list = []
        self.time_list = []
        self.step_list = []
        self.function_log_list = []
        self.queue_list = []
       

    def route(self):
        
        start_time = time.time()

        #start position
        c=list(dict_start_points.values())
        startpoint_node = np.random.choice(c, 1)[0]


        #input for the function
        node_from_here = startpoint_node
        startpoint = startpoint_node
        total_round = self.total_round
        dict_attractions = dict_waypoints_pleasurecraft
        copy_w_G_totaal= scenario_G.copy() # keeps track on path actually visited as input for next iteration


        #output
        total_path =[]
        

        def route_planner_captain(node_from_here, total_path, startpoint, total_round, dict_attractions, copy_w_G_totaal):

            penalty = 2
            treshold = 100
            reference_total_round = total_round

            copy_sightseeing_dict = copy.deepcopy(dict_attractions)

            minimal_distance_next_attraction = 100000000000000
            final_distance_back = 0
            final_path_back =[]
            
            path = []
            
            dict_zones_copy = copy.deepcopy(dict_zones)
            
            list_names=[]
            list_nodes=[]
            self.next_point_zone =[]
            for i,v in zip(dict_zones_copy.values(), dict_zones_copy.keys()):
                list_nodes.append(i)
                list_names.append(v)

                
            j=0
            while total_round - final_distance_back > treshold:
                
                #line of code makes sure no neigboring nodes are drawn in order to prevent turn around.
                #makes copy of dictionary 
                copy_sightseeing_dict_for_drawing = copy.deepcopy(copy_sightseeing_dict)


                
#                 print(list_nodes)
#                 if len(list_nodes) !=
#                 if list_names[j] == 'redlightdistrict' or list_names[j] == 'zone ;4 centrum':
#                     list_checker = [x for x in list_nodes if x != [] and x != list_nodes[5]]
#                     print("list checker if", list_names[j], list_names[3], list_names[5])

#                 elif list_names[j] == 'zone 1; links boven':
#                     list_checker = [x for x in list_nodes if x != [] and x != list_nodes[3] and  x != list_nodes[4]]
#                     print("list checker elif", list_checker)

                list_checker = [x for x in list_nodes if x != []]
    
                if total_round == reference_total_round:
              
                    lis_nodes_not=[]
                    for i in c:
                        p = list(w_G.successors(i))
                        x = list(w_G.predecessors(i))
                        if len(p) == 1 and len(x) == 1 and p[0] == x[0]:
                            lis_nodes_not.append(i)
                        list_nodes =[i for i in list_checker if i not in lis_nodes_not]
                        next_point_zone = random.sample(list_nodes, 1)[0]
                        next_point_node= random.sample(next_point_zone, 1)
                else:

                    list_nodes = list_checker
                    next_point_zone = random.sample(list_nodes, 1)[0]
                    next_point_node= random.sample(next_point_zone, 1)
          
        
                neighbors = [n for n in copy_w_G_totaal[next_point_node[0]]]
                
                
                def possibillity_checker(next_point_node, node_from_here, startpoint, copy_w_G_totaal, path):
                    
                    w_G_path =scenario_G.copy()
                    
                    if total_round < reference_total_round and w_G_path.has_edge(path[len(path)-1], path[len(path)-2]) == True:
                        w_G_path.remove_edge(path[len(path)-2], path[len(path) -1])
                        try:
                            w_G_path.remove_edge(path[len(path)-1], path[len(path) -2])
                        except Exception:
                            pass

                    if nx.has_path(w_G_path, node_from_here, next_point_node[0] )== True:
                        
                        path_1 = nx.dijkstra_path(w_G_path, node_from_here, next_point_node[0], weight = 'dis')
                        
                        
                        if total_round < reference_total_round and w_G_path.has_edge(path_1[len(path_1)-1], path_1[len(path_1)-2]) == True:
                            w_G_path.remove_edge(path_1[len(path_1)-2], path_1[len(path_1) -1])
                            try:
                                w_G_path.remove_edge(path_1[len(path_1)-1], path_1[len(path_1) -2])
                            except Exception:
                                pass
                            
                        path_check = nx.has_path(w_G_path, next_point_node[0], startpoint)
                        
                    else:
                        path_check=False
                     
                    return path_check
                
                
                possible_path = possibillity_checker(next_point_node, node_from_here, startpoint, copy_w_G_totaal, path)
                            
                is_path = nx.has_path(copy_w_G_totaal, node_from_here, next_point_node[0])
                
                while is_path == False or copy_w_G_totaal.edges[next_point_node[0], neighbors[0]]['visited'] > 0\
                or possible_path == False:
                    list_checker = [x for x in list_nodes if x != []]
                    list_nodes = list_checker
                    if len(list_nodes)==0:
                        break
                    next_point_zone = random.sample(list_nodes, 1)[0]
                    next_point_node= random.sample(next_point_zone, 1)
                    is_path = nx.has_path(copy_w_G_totaal, node_from_here, next_point_node[0])
                    possible_path = possibillity_checker(next_point_node, node_from_here, startpoint, copy_w_G_totaal, path)
                    neighbors = [n for n in copy_w_G_totaal[next_point_node[0]]]
    
                    j = list_nodes.index(next_point_zone)
                    list_nodes[j].remove(next_point_node[0])
        
            
                if len(list_nodes)==0:
                    total_round -= final_distance_back
                    for i in final_path_back:
                        total_path.append(i)
                    break
 
               
                
#                 j = list_nodes.index(next_point_zone)
#                 print("list_nodes", list_nodes, "index", j, "next_point_node", next_point_node )
#                 list_nodes[j].remove(next_point_node[0])
                
                j = list_nodes.index(next_point_zone)
                list_nodes.remove(next_point_zone)
                

                for attraction in next_point_node:
                    
                    copy_w_G_1 = copy_w_G_totaal.copy()
                    
                    #two edges
                    if total_round < reference_total_round and copy_w_G_1.has_edge(path[len(path)-2], path[len(path)-1]) == True\
                    and copy_w_G_1.has_edge(path[len(path)-1],path[len(path)-2]) == True:
                        copy_w_G_1.remove_edge(path[len(path)-2], path[len(path) -1])
                        try:
                            copy_w_G_1.remove_edge(path[len(path)-1], path[len(path) -2])
                        except Exception:
                            pass
                                  
                    distance_attraction = nx.dijkstra_path_length(copy_w_G_1, node_from_here, attraction , weight = 'dis')
                    path_next_attraction = nx.dijkstra_path(copy_w_G_1, node_from_here, attraction, weight = 'dis') 
                    
                    distance_attaction= 0
                    for i in range(len(path_next_attraction)-1):
                        copy_w_G_1.edges[path_next_attraction[i], path_next_attraction[i+1] ] ['dis'] *= penalty
                        try:
                            copy_w_G_1.edges[path_next_attraction[i+1], path_next_attraction[i] ] ['dis'] *= penalty
                        except Exception:
                            pass
                        

                    ### om te voorkomen dat hij op het einde niet terug keert, werkt nog niet helemaal; paar uitzonderingen vandaar try except
                     #print("path_next_attraction", path_next_attraction)
            
                    if total_round < reference_total_round and copy_w_G_1.has_edge(path_next_attraction[len(path_next_attraction)-1],path_next_attraction[len(path_next_attraction)-2]) == True:
                        copy_w_G_1.remove_edge(path_next_attraction[len(path_next_attraction)-2], path_next_attraction[len(path_next_attraction) -1])
                        try:
                            copy_w_G_1.remove_edge(path_next_attraction[len(path_next_attraction)-1], path_next_attraction[len(path_next_attraction) -2])
                        except Exception:
                            pass
                    
                    
                    distance_attraction_back_home = nx.shortest_path_length(copy_w_G_1, attraction, startpoint, weight = 'dis')
                    path_back_home = nx.shortest_path(copy_w_G_1, attraction, startpoint, weight = 'dis')

                    if distance_attraction + distance_attraction_back_home < total_round:
                       
                        total_round -= distance_attraction
                        path = path_next_attraction
                        node_from_here  = attraction
                        final_path_back  = path_back_home
                        final_distance_back = distance_attraction_back_home
                    
                        for i in range(len(path)-1):
                            copy_w_G_totaal.edges[ path[i], path[i+1] ] ['dis'] *= penalty
                            copy_w_G_totaal.edges[ path[i], path[i+1] ] ['visited'] +=1
                            try:
                                copy_w_G_1.edges[path[i+1], path[i] ] ['dis'] *= penalty
                                copy_w_G_totaal.edges[ path[i+1], path[i] ] ['visited'] +=1
                            except Exception:
                                pass
              

                            

                        #add path to total path
                        for i in path:
                            total_path.append(i)
                            #staan nog dubbele tussen
             
                if total_round - final_distance_back < treshold:
                    #hier heeft ie de laatste attractie nog niet weggehaald, wat wel gebeurd bij de vorige if
                    total_round -= final_distance_back
                    for i in final_path_back:
                        total_path.append(i)
                    break  



                elif len(list_nodes)==0:
                    total_round -= final_distance_back
              
                    for i in final_path_back:
                        total_path.append(i)
                    break 
                    
          


            return total_path#, total_percentage
    
        results= route_planner_captain(node_from_here, total_path, startpoint, total_round, dict_attractions, copy_w_G_totaal)            
        route = results  
        


        return route

## Scenario's

In [12]:
#july

druktelijsten= pd.read_excel('druktelijst.xlsx')
druktelijst_rcv = druktelijsten.loc['druktelijst_rcv'].values.tolist()
druktelijst_cv = druktelijsten.loc['druktelijst_cv'].values.tolist()
druktelijst_pc = druktelijsten.loc['druktelijst_pc'].values.tolist()

growthfactor_list = [2.3]
for growthfactor in growthfactor_list:

    druktelijst_rcv = [int(i * growthfactor) for i in druktelijst_rcv]
    druktelijst_cv = [int(i * growthfactor) for i in druktelijst_cv]
    druktelijst_pc = [int(i * growthfactor) for i in druktelijst_pc]


FileNotFoundError: [Errno 2] No such file or directory: 'druktelijst.xlsx'

## Simulation Environment

In [17]:
def distr_gen_from_distr_list(druktelijst):
        start_time = time.time()
        
        te_returnen_lijst = []
        for uurdrukte in druktelijst:

            uurlijst = list(np.zeros(round(sim_steps/len(druktelijst)), dtype = int))
            for i in range(uurdrukte):
                uurlijst[random.choice(range(len(uurlijst)))] += 1
            te_returnen_lijst += uurlijst
     
        if sim_steps > len(te_returnen_lijst):
            for i in range(sim_steps - len(te_returnen_lijst)):
                te_returnen_lijst.append(0)
                
        elif len(te_returnen_lijst) > sim_steps:
            te_returnen_lijst = te_returnen_lijst[:len(te_returnen_lijst) - (len(te_returnen_lijst) - sim_steps)]
            
        return te_returnen_lijst
    


#-----------------Start simulation------------------------------------------------#   
      

#simulation environment
env = simpy.Environment()

#crossings; adds resource
server_list =[]
not_server_list =[285, 236, 259, 159, 322, 334, 355, 361, 364, 365, 372, 15, 26]
server_list_5 = [39, 52, 82, 2]
server_list_3_add =[296, 280, 249, 201, 142, 121, 85, 31, 20]
server_list_2 = [20, 351, 338, 331, 330]


for i in list(w_G.nodes):
    neighbors = [n for n in w_G[i]] 
    if len(neighbors)>2 and i not in not_server_list :
        server_list.append(i)
for i in server_list_3_add:
    server_list.append(i)
    
resource_list = []
for i in server_list:
    resource_list.append(i)

dict1 ={}
for i, v in zip(resource_list, server_list) :
    if i in server_list_5:
        i = simpy.Resource(env, capacity = 5)
    elif i in server_list_2:
        i = simpy.Resource(env, capacity = 2)
    else:
        i = simpy.Resource(env, capacity = 3)
    dict1[v] = i
    
    
    
dict_queuelength = {}


for i in dict1.keys():
    dict_queuelength[i] = []

def observe(env):
    while True:
        obs_time.append(env.now)
        for i in dict1.keys():
            dict_queuelength[i].append(len(dict1[i].queue))
#         q_length.append(len(dict1[127].queue))
        yield env.timeout(1)#it timesout the environment between succesive obeservations  
    
#adds obeserver to env  
env.process(observe(env))   
    
#startvariables of simulation 
ves_list =[]
start_time_list =[]
step_count = 0
step_iter = 0
stepsize = 10 #in seconds
sim_steps =int((24*60*60)/stepsize)
c = list(w_G.nodes)
distr_rcv = distr_gen_from_distr_list(druktelijst_rcv)
distr_pc = distr_gen_from_distr_list(druktelijst_pc)
distr_cv = distr_gen_from_distr_list(druktelijst_cv)
total_matrix_len = 0
obs_time = []
vessel_counter = 0

#calculates simulation time
import time
start_time = time.time()

current_time = time.strftime("Simpy_%Y_%m_%d___%H_%M_%S")
csv_name = current_time + '.csv'
print(csv_name)

with open(str(csv_name), "w", newline='') as csv_file:
    writer = csv.writer(csv_file, delimiter = ',')
    #actual simulation; start
    for i in range(1, sim_steps):
        env.run(until=i)
        print(i)

        for y in range(distr_rcv[step_count]):
            ves_list.append(Random_Cruise_Vessel(env, stepsize, step_iter, vessel_counter))
            vessel_counter += 1
            start_time_list.append(step_count)

        for z in range(distr_pc[step_count]):
            ves_list.append(Pleasure_Craft(env, stepsize, step_iter, vessel_counter))
            vessel_counter += 1
            start_time_list.append(step_count)

        for x in range(distr_cv[step_count]):
            ves_list.append(Cruise_Vessel(env, stepsize, step_iter, vessel_counter))
            vessel_counter += 1
            start_time_list.append(step_count) # Generate list of starttimes whe


        step_count += 1
        step_iter += 1

print("simulation took {} s to complete".format(time.time() - start_time))
print("simulation took {} min to complete".format((time.time() - start_time)/60))

Simpy_2018_12_18___23_10_34.csv
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
324
323
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267


1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923


KeyError: 'the label [162] is not in the [index]'

## Animator + Plot of routes


In [13]:
# %matplotlib notebook
# fig = plt.figure(figsize=(10,10))
# nx.draw(w_G,nx.get_node_attributes(w_G, 'pos'), node_size =1)
# plt.scatter(simulation.x_df,simulation.y_df)
# #plt.savefig('/Users/JeroenvanderDoes/Downloads/Amsterdam_Canals_11.pdf', format='pdf', rasterized=False)
# plt.show()


In [14]:
# print(server_list)

In [15]:
# %matplotlib notebook
# %matplotlib notebook
# average_wait_time_list = []

# for ves in ves_list:
#     average_wait_time_list.append(ves.total_wait_time)
# print(len(average_wait_time_list))
    
# import matplotlib.pyplot 
# plt.figure(i, figsize =(5,5))
# plt.hist(average_wait_time_list, bins = 20)
# plt.xlabel("waiting time (steps)")
# plt.ylabel("Number of vessels")


In [16]:
# for ves in ves_list:
#     if ves.total_wait_time > 2500:
#         print("path","begin", ves.path[0] , "end", ves.path[len(ves.path)-1])

In [17]:
# %matplotlib notebook
# %matplotlib notebook
# import matplotlib.pyplot 


# for i in dict_queuelength:
# #     if i == 10 or i == 318:
#     plt.figure(i, figsize =(5,5))
#     plt.step(obs_time, dict_queuelength[i], where = 'post')
#     plt.xlabel("Time steps of simulation")
#     plt.ylabel('Queue length of node %d' %i)

In [18]:
# %matplotlib notebook
# %matplotlib notebook
# import matplotlib.pyplot as plt
# i = 0
# for ves in ves_list[:20]:
#     plt.figure(i, figsize =(5,5))
#     nx.draw(scenario_G_draw,nx.get_node_attributes(scenario_G_draw, 'pos'), node_size =1)
#     plt.scatter(ves.coor_log_x,ves.coor_log_y)
#     i+=1
# #     plt.xlabel("Time steps of simulation")
# #     plt.ylabel('Queue length of node %d' %i)

In [19]:
# # Create pandas dataframe of vessels log' for plotting purposes.
# x_df = pd.DataFrame(np.zeros([sim_steps, len(ves_list)]))
# y_df = pd.DataFrame(np.zeros([sim_steps, len(ves_list)]))
# #self.df = pd.DataFrame(np.zeros([self.sim_steps, len(self.ves_list)]))# generate dataframe based on rows: # steps ; colums: # vessels
# #self.from_df = pd.DataFrame(np.zeros([self.sim_steps, len(self.ves_list)]))

# for i, time, ves in zip(range(len(ves_list)), start_time_list, ves_list):
#     #creates dataframe for x coordinates and one for y coordinates based on vessel logs
#     ds_x = pd.Series(np.array(ves.coor_log_x, dtype = np.float64))
#     ds_y = pd.Series(np.array(ves.coor_log_y, dtype = np.float64))

#     x_df.at[time : time + len(ds_x) -1, i] = ds_x.values
#     y_df.at[time : time + len(ds_y) -1, i] = ds_y.values

#     #replaces values of 0 to nan for plotting purposes
#     x_df = x_df.replace(0.0, np.nan) 
#     y_df = y_df.replace(0.0, np.nan)

In [20]:
# import time
# start_time = time.time()

# def get_data_x(i):
#     return x_df.loc[i]

# def get_data_y(i):
#     return y_df.loc[i]

# def animate(i):
#     x = get_data_x(i)
#     y= get_data_y(i)   
#     graph.set_data(x,y)
#     return graph,

# # Set up formatting for the movie files
# Writer = animation.writers['ffmpeg']
# writer = Writer(fps=15, metadata=dict(artist='Me'), bitrate=1800)

# fig = plt.figure(figsize=(20,20))
# ax = plt.axes()

# nx.draw(scenario_G_draw,nx.get_node_attributes(scenario_G_draw, 'pos'),node_size = 0.1)

# n_steps = x_df.index 

# graph, = plt.plot([],[],'o')

# ani = animation.FuncAnimation(fig, animate, frames= n_steps, interval=1, repeat=True, blit = True)
# ani.save('/Users/JeroenvanderDoes/Downloads/simpy_v5.mp4', writer=writer)

# print("Animator : {} s".format(time.time() - start_time))
